In [ ]:
###PLEASE NOTE
##we used some of the data loading/saving code from here:
##Fierro, Miguel, Ye Xing, and Tao Wu. "Quick-Start Guide to the Data Science Bowl Lung Cancer Detection Challenge, Using Deep Learning, Microsoft Cognitive Toolkit and Azure GPU VMs." Blog post. Cortana Intelligence and Machine Learning Blog. Microsoft, 17 Feb. 2017. Web. 10 Mar. 2017.
##this is the short code to load the features from file and make an array per patient



devtype = 1 #1 for gpu, 0 for cpu

import sys
import os

import numpy as np
import pandas as pd
import itertools
import scipy
import random


from cntk import Trainer, Axis
from cntk.io import MinibatchSource, CTFDeserializer, StreamDef, StreamDefs,\
        INFINITELY_REPEAT
from cntk.learners import sgd, learning_rate_schedule, UnitType
from cntk import input, cross_entropy_with_softmax, \
        classification_error, sequence
from cntk.logging import ProgressPrinter
from cntk.layers import Sequential, Embedding, Recurrence, LSTM, GRU, Dense, \
        BatchNormalization, Dropout, For, Convolution, Fold
from cntk.io import UserMinibatchSource, StreamInformation, MinibatchData
from cntk.ops.functions import Function

import cntk as C

from __future__ import print_function
from cntk.layers import *
from cntk.layers.sequence import *
from cntk.layers.models.attention import *
from cntk.layers.typing import *




if devtype ==1:
    C.device.try_set_default_device(C.device.gpu(0))
else:
    C.device.try_set_default_device(C.device.cpu())



##this grouper function is just from SO
def grouper(n, iterable):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk

In [ ]:
EXPERIMENT_NUMBER = '001' 

#Put here the path where you downloaded all kaggle data
DATA_PATH='/a/h/cole/data/'

# Path and variables
STAGE1_LABELS=DATA_PATH + 'stage1_labels.csv'
STAGE1_SAMPLE_SUBMISSION=DATA_PATH + 'stage1_sample_submission.csv'
STAGE1_SAMPLE_SUBMISSION_SOLUTIONS=DATA_PATH + 'stage1_solution.csv'
STAGE1_FOLDER=DATA_PATH + 'stage1/'
FEATURE_FOLDER=DATA_PATH + 'features/features' + EXPERIMENT_NUMBER + '/'
SUBMIT_OUTPUT='submit' + EXPERIMENT_NUMBER + '.csv'


In [ ]:
def make_two_classes(y):
    ys=[]
    for label in y:
        if label:
            ys.append((1,0))
        else:
            ys.append((0,1))
    return np.array(ys).astype(np.float32)

df = pd.read_csv(STAGE1_LABELS)

# x = [np.squeeze(np.load(FEATURE_FOLDER+'%s.npy' % str(id))) for id in df['id'].tolist()]

xresnet = [np.squeeze(np.load(FEATURE_FOLDER+'%s.npy' % str(id))) for id in df['id'].tolist()]
xvgg = [np.squeeze(np.load('/a/data/mango/featuresvgg19/'+'%s.npy' % str(id))) for id in df['id'].tolist()]

x=[]
for r,v in zip(xresnet,xvgg):
    v = v.reshape((v.shape[0],-1))
    x.append(np.concatenate((r,v),axis=1))


#gives python list of length 1397, each item has sizes like (88, 2048) or (77, 2048)
#x = np.array(x)
y = df['cancer'].as_matrix()
# print y.shape (1397,)

#x_train, x_val, y_train, y_val = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
#                                                                       test_size=0.20)
y = make_two_classes(y)

# xm = []
# for xi in x:
#     xm.append(xi / xi.shape[0])
    
# x = xm

In [ ]:
# import cPickle
# # cPickle.dump(x, open('x_test_resvgg.pickle', 'wb')) 
# # cPickle.dump(y, open('y_test_resvgg.pickle', 'wb')) 

# x = cPickle.load(open('x_test_resvgg.p', 'rb'))
# y = cPickle.load(open('y_test_resvgg.p', 'rb'))

In [ ]:
print (x[0].shape)

In [ ]:
input_dim = x[0].shape[1]


#had to fight with cntk... custom batch source for our numpy data
class MyDataSource(UserMinibatchSource):
    def __init__(self, f_dim, l_dim):
        #pass
        self.f_dim, self.l_dim = f_dim, l_dim

        self.fsi = StreamInformation("features", 0, 'dense', np.float32, (self.f_dim,))
        self.lsi = StreamInformation("labels", 1, 'dense', np.float32, (self.l_dim,))

#        self.data_iter = grouper(minibatch_size,x)
#        self.label_iter = grouper(minibatch_size,y)
        
#         print("a")
        super(MyDataSource, self).__init__()

    #this whole function is to work around a bug in cntk wtf
    def turn_var_into_value_wtf(self,var,data):
        data = [C.Value._as_best_data_type(var, sample) for sample in data]
        borrow = devtype == C.DeviceKind.CPU
        list_of_ndavs = [C.NDArrayView.from_data(sample, device=C.cpu(),
                                               borrow=borrow)
                         for sample in data]

        from cntk.internal import sanitize_shape
        value = C.cntk_py.Value_create(
            sanitize_shape(var.shape),
            list_of_ndavs,
            [],
            C.use_default_device(),
            False,
            True)  # always create a copy in Value
        return value
    
    def stream_infos(self):
#        pass
#        print("b")
        return [self.fsi, self.lsi]

    def next_minibatch(self, num_samples, number_of_workers=1, worker_rank=0, device=None):
#        pass
#        print("c")
#         try:
#             train_features = list(next(self.data_iter))
#             labels = list(next(self.label_iter))
#         except StopIteration:
#             #continue
#             self.data_iter = grouper(minibatch_size,x)
#             self.label_iter = grouper(minibatch_size,y)
#             train_features = list(next(self.data_iter))
#             labels = list(next(self.label_iter))
        
        idxs = random.sample(xrange(len(x)), num_samples)
        train_features = [x[i] for i in idxs]
        labels = [y[i] for i in idxs]
    
        sweep_end = False
        
        num_seq = len(train_features)
        f_sample_count = num_seq
        l_sample_count = num_seq
        
        
        dummyvar = sequence.input(shape=input_dim, is_sparse=False)        
        f_data = self.turn_var_into_value_wtf(dummyvar,train_features)
        l_data = C.Value(batch=np.asarray(labels, dtype=np.float32))

        
        result = {
#                 self.fsi: train_features,
#                 self.lsi: labels
                self.fsi: MinibatchData(f_data, num_seq, f_sample_count, sweep_end),
                self.lsi: MinibatchData(l_data, num_seq, l_sample_count, sweep_end)
               
                }


        return result

In [ ]:
def gettestrnn():
    #factor = C.constant([)
    def testrnn(prev_h,prev_s,x):
            #dhs = Sdh(dh)  # previous value, stabilized
            #ht = activation (times(x, W) + times(dhs, H) + b)
            #h = times(Sht(ht), Wmr) if has_projection else \
            #    ht
            #s = C.reduce_mean(x)
            x_mean = C.reduce_mean(x)
            x_one_if_high = C.greater(x,x_mean)
            sum_high = C.reduce_sum(C.element_times(x,x_one_if_high))
            count_high = C.reduce_sum(x_one_if_high)
            three_quarter_mark = sum_high/count_high
            
            upper_count = C.reduce_sum(C.greater(x,three_quarter_mark))
            s = (prev_s + prev_s + upper_count)/3
            #ADD CLIPPING TODO
            #h = (x * C.sigmoid(s)) 
            h = (x * s) 
            h += ((prev_h/prev_h) - 1)
            #h *= 0
            #s *= 0
            #return Function.NamedOutput(h=h)
            return h,s
    return BlockFunction('testrnn', '')(testrnn)

def gettestrnn2():
    #factor = C.constant([)
    def testrnn(prev_h,prev_sum,prev_counter,x):
            x_mean = C.reduce_mean(x)
            x_one_if_high = C.greater(x,x_mean)
            sum_high = C.reduce_sum(C.element_times(x,x_one_if_high))
            count_high = C.reduce_sum(x_one_if_high)
            this_high = C.greater(count_high,0)
  
            the_sum = C.minus(C.plus(prev_sum,x),x)
            s_counter = (prev_counter + count_high - count_high)


            h = C.clip(x,0,0) * the_sum * s_counter 
            h += (C.clip(prev_h,0,0))
            
                    

            if C.equal(s_counter,0):
                the_sum += x
                if this_high:
                    s_counter += 2
                    
            else:
                if this_high:
                    the_sum += C.plus(prev_sum,x)
                    s_counter += 1
                else:
                    s_counter -= s_counter
                    the_sum += x
                    if C.greater(s_counter,4):
                        pass
                        h += C.element_divide(the_sum,s_counter)
            h = C.reshape(h,(1024))
#             the_sum = C.reshape(the_sum,(1))
#             s_counter = C.reshape(s_counter,(1))
            
            return h,the_sum,s_counter
                       


    return BlockFunction('testrnn2', '')(testrnn)

from cntk.internal import sanitize_dtype_cntk
zero_cons = Constant.scalar(sanitize_dtype_cntk(np.float32), 0.0)

def get_mean_rnn():
    #factor = C.constant([)
    def mean_rnn(mean,sum_r,counter,x):
        #sum_r = C.clip(sum_r,0,0) + x
        sum_r += x
        
        count_high = C.reduce_sum(x)
        counter = counter + 1 + (count_high - count_high)
        #counter += 1
        mean = C.clip(mean,0,0)
        
        if C.ops.equal(C.sequence.future_value(x),zero_cons): #we are at the last slice
            mean += (sum_r / counter)
        else:
            mean += x
        
        return mean, sum_r, counter
                       
    return BlockFunction('mean_rnn', '')(mean_rnn)

def get_upper_mean_rnn():
    #factor = C.constant([)
    def upper_mean_rnn(reduced_mean_storage,sum_r,counter,x):
        sum_r += C.clip(x,0,0)
        count_high = C.reduce_sum(x)
        counter = counter + (count_high - count_high)
        reduced_mean_storage += C.clip((sum_r / counter),0,0)
 
        
        if C.ops.equal(C.sequence.past_value(x),zero_cons): #we are at first slice
            reduced_mean_storage += C.reduce_mean(x)
        else:
            reduced_mean = C.reduce_mean(reduced_mean_storage)
            if C.ops.greater(C.reduce_mean(x),reduced_mean):
                sum_r += x
                counter = counter + 1
            if C.ops.equal(C.sequence.future_value(x),zero_cons): #we are at the last slice
                reduced_mean_storage -= reduced_mean_storage #reset to zero
                reduced_mean_storage += (sum_r / counter)

        
        return reduced_mean_storage, sum_r, counter
                       
    return BlockFunction('upper_mean_rnn', '')(upper_mean_rnn)

def get_upper_mean_rnn2():
    #factor = C.constant([)
    def upper_mean_rnn2(mean,sum_r,counter,x):
        sum_r += C.clip(x,0,0)
        count_high = C.reduce_sum(x)
        counter = counter + (count_high - count_high)
        mean += C.clip((sum_r / counter),0,0)

 
        
        if C.ops.equal(C.sequence.past_value(x),zero_cons): #we are at first slice
            mean += x
        else:
            real_high_count = C.reduce_sum(C.greater(x,mean))
            if C.ops.greater(real_high_count,75):
                sum_r += x
                counter = counter + 1
            if C.ops.equal(C.sequence.future_value(x),zero_cons): #we are at the last slice
                mean -= mean #reset to zero
                mean += (sum_r / counter)

        
        return mean, sum_r, counter
                       
    return BlockFunction('upper_mean_rnn2', '')(upper_mean_rnn2)


In [ ]:
#below function should make last slice a count of slices...
#but it kills training, loss never gets low. bad code? cntk issue? who knows...
def get_counter_rnn():
    #factor = C.constant([)
    def counter_rnn(prev_h,counter,x):
        
        
        count_high = C.reduce_sum(x)
        counter = counter + 1 + (count_high - count_high)
        #counter += 1
        h = (prev_h-prev_h)+(x-x)
        
        if C.ops.equal(C.sequence.future_value(x),zero_cons): #we are at the last slice
            h += counter
            h = C.reciprocal(h)
        else:
            h += x
        
        
        return h, counter
                       
    return BlockFunction('counter_rnn', '')(counter_rnn)


def get_scale_rnn():
    #factor = C.constant([)
    def scale_rnn(prev_h,scale,x):
        scale += (prev_h-prev_h)
        
        if C.ops.equal(C.sequence.past_value(x),zero_cons): #we are at the first slice
            scale += x
        
        h = x*scale
        
        return h, scale
                       
    return BlockFunction('scale_rnn', '')(scale_rnn)


#tried to modify mgru with some sequence size info
#however could never reliably get access to the sequence size in the first place
def get_mgru(shape):
    '''
    Helper to create a recurrent block of type 'LSTM', 'GRU', or RNNUnit.
    '''
    activation = C.ops.tanh
    init = C.initializer.glorot_uniform()
    init_bias = 0
    has_projection = False
    
    _INFERRED = (C.InferredDimension,) 

    shape = C.internal._as_tuple(shape)

    cell_shape = shape
    if len(shape) != 1 or len(cell_shape) != 1:
        raise ValueError("%s: shape and cell_shape must be vectors (rank-1 tensors)" % type)
        # otherwise we'd need to fix slicing and Param initializers

    stack_axis = -1  # for efficient computation, we stack multiple variables (along the fastest-changing one, to match BS)
    # determine stacking dimensions
    cell_shape_list = list(cell_shape)
    stacked_dim = cell_shape_list[stack_axis]
    cell_shape_list[stack_axis] = stacked_dim * 3
    cell_shape_stacked = tuple(cell_shape_list)  # patched dims with stack_axis duplicated 4 times
    cell_shape_list[stack_axis] = stacked_dim * 2
    cell_shape_stacked_H = tuple(cell_shape_list)  # patched dims with stack_axis duplicated 4 times

    # parameters
    b  = C.Parameter(            cell_shape_stacked,   init=init_bias, name='b')                              # bias
    W  = C.Parameter(_INFERRED + cell_shape_stacked,   init=init,      name='W')                              # input
    H  = C.Parameter(shape     + cell_shape_stacked_H, init=init,      name='H')                              # hidden-to-hidden
    H1 = C.Parameter(shape     + cell_shape,           init=init,      name='H1') # hidden-to-hidden
    H2 = C.Parameter(shape,           init=init,      name='H2') # new
    Ci = None  # cell-to-hiddden {note: applied elementwise}
    Cf = None  # cell-to-hiddden {note: applied elementwise}
    Co = None  # cell-to-hiddden {note: applied elementwise}

    Wmr = C.Parameter(cell_shape + shape, init=init, name='P') if has_projection else None  # final projection

    # each use of a stabilizer layer must get its own instance
    Sdh = C.layers.blocks.Stabilizer(enable_self_stabilization=True, name='dh_stabilizer')

    # define the model function itself
    # general interface for Recurrence():
    #   (all previous outputs delayed, input) --> (outputs and state)
    # where
    #  - the first output is the main output, e.g. 'h' for LSTM
    #  - the remaining outputs, if any, are additional state
    #  - if for some reason output != state, then output is still fed back and should just be ignored by the recurrent block

    
    # GRU model function
    # in this case:
    #   (dh, x) --> (h)
    # e.g. https://en.wikipedia.org/wiki/Gated_recurrent_unit
    def grun(dh,rec_length_of_seq, x):
        
        if C.ops.equal(C.sequence.past_value(x),zero_cons): #we are at first slice
            rec_length_of_seq += C.reduce_min(x)
        
        dhs = Sdh(dh)  # previous value, stabilized
        # note: input does not get a stabilizer here, user is meant to do that outside

        # projected contribution from input(s), hidden, and bias
        projx3 = b + C.times(x, W)
        projh2  = C.times(dhs, H)

        zt_proj = C.slice (projx3, stack_axis, 0*stacked_dim, 1*stacked_dim) + C.slice (projh2, stack_axis, 0*stacked_dim, 1*stacked_dim)
        rt_proj = C.slice (projx3, stack_axis, 1*stacked_dim, 2*stacked_dim) + C.slice (projh2, stack_axis, 1*stacked_dim, 2*stacked_dim)
        ct_proj = C.slice (projx3, stack_axis, 2*stacked_dim, 3*stacked_dim)

        zt = C.sigmoid (zt_proj)        # update gate z(t)

        rt = C.sigmoid (rt_proj)        # reset gate r(t)

        rs = dhs * rt        # "cell" c
        ct = activation (ct_proj + times(rs, H1))

        
        #new
        weighted_los = C.sigmoid(times(rec_length_of_seq,H2))
        
        ht = (1 - zt) * ct + zt * ((1-weighted_los) * dhs) # hidden state ht / output

        # for comparison: CUDNN_GRU
        # i(t) = sigmoid(W_i x(t) +          R_i h(t-1)  + b_Wi + b_Ru)
        # r(t) = sigmoid(W_r x(t) +          R_r h(t-1)  + b_Wr + b_Rr)   --same up to here
        # h'(t) =   tanh(W_h x(t) + r(t) .* (R_h h(t-1)) + b_Wh + b_Rh)   --r applied after projection? Would make life easier!
        # h(t) = (1 - i(t) .* h'(t)) + i(t) .* h(t-1)                     --TODO: need to confirm bracketing with NVIDIA

        h = C.times(Sht(ht), Wmr) if has_projection else \
            ht

        # returns the new state as a tuple with names but order matters
        #return Function.NamedOutput(h=h)
        return h, rec_length_of_seq

    # return the corresponding lambda as a CNTK Function
    return BlockFunction('grun', 'grun')(grun)

In [ ]:
# def generate_random_data(sample_size, feature_dim, num_classes):
#     # Create synthetic data using NumPy.
#     Y = np.random.randint(size=(sample_size, 1), low=0, high=num_classes)
#     S = np.random.randint(size=(sample_size), low=10, high=30)

#     # Make sure that the data is separable
#     X = []
#     for i in range(sample_size):
#         x = (np.random.randn(S[i], feature_dim) + 3) * (Y[i]+1)
#         x = x.astype(np.float32)
#         X.append(x)
#     # converting class 0 into the vector "1 0 0",
#     # class 1 into vector "0 1 0", ...
#     class_ind = [Y == class_number for class_number in range(num_classes)]
#     Y = np.asarray(np.hstack(class_ind), dtype=np.float32)
#     return X, Y


# Defines the STM model for classifying sequences
def LSTM_sequence_classifier_net(input, num_output_classes, LSTM_dim, cell_dim):
    lstm_classifier = Sequential([
        #For(range(3),lambda: [Dense(2048, activation=C.leaky_relu), Dropout(0.4)]),
        
#         Dense(2048, activation=C.leaky_relu),
        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        #Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        #BatchNormalization(),
        
        
        #Fold(C.reduce_max),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
        
        Fold(get_mean_rnn(),initial_state=(0,0,0)),
        
#         Dropout(0.2),
        #Recurrence(get_counter_rnn(),initial_state=(0,0)),  
        #Recurrence(get_scale_rnn(),initial_state=(0,0),go_backwards=True),  
        #Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        #Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        #Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
        #LayerNormalization(),
        #Recurrence(gettestrnn2(),initial_state=(0.0001,0,0)),
        #BatchNormalization(),
#         Recurrence(get_counter_rnn(),initial_state=(0,0)),
#         Recurrence(get_mgru(LSTM_dim),initial_state=(0,1),go_backwards=True),
#         Recurrence(get_counter_rnn(),initial_state=(0,0)),
#         Fold(get_mgru(LSTM_dim),initial_state=(0,1),go_backwards=True),
        #Recurrence(get_counter_rnn(),initial_state=(0,0),go_backwards=True),
        
        #Fold(get_upper_mean_rnn(),initial_state=(0,0,0),go_backwards=True),
        #Fold(get_sum_rnn(),initial_state=(0,0)),
#         Fold(C.plus),
        #Dropout(0.3),
        #Fold(LSTM(LSTM_dim, cell_dim, enable_self_stabilization=True,use_peepholes=True)),
#         sequence.last,
        #BatchNormalization(),
        #Dropout(0.3),
        Dense(num_output_classes)])
        
    return lstm_classifier(input)


cell_dim = 100
hidden_dim = 100
num_output_classes = 2

features = sequence.input(shape=input_dim, is_sparse=False)
label = input(num_output_classes)

#def make_trainer():


    # Input variables denoting the features and label data


    # Instantiate the sequence classification model
model = LSTM_sequence_classifier_net(
    features, num_output_classes, hidden_dim, cell_dim)

ce = cross_entropy_with_softmax(model, label)
#ce = C.squared_error(model, label)
#ce = C.binary_cross_entropy(model, label)
#pe = classification_error(model, label)

lr_per_sample = learning_rate_schedule(0.001, UnitType.sample)
m_per_sample = C.momentum_schedule(0.9, UnitType.sample)
# Instantiate the trainer object to drive the model training
progress_printer = ProgressPrinter(0)
#trainer = Trainer(model, (ce, pe),
trainer = Trainer(model, ce,
                  #sgd(model.parameters, lr=lr_per_sample),
                  C.adam(model.parameters, lr_per_sample, m_per_sample),
                  progress_printer)
#return trainer

#def train_trainer(trainer):
    # Get minibatches of sequences to train with and perform model training




mbs = MyDataSource(input_dim, num_output_classes)
model_inputs_to_streams = {
    features: mbs.fsi,
    label: mbs.lsi
}


checkpoint_config=C.train.CheckpointConfig("cntk_ts_checkpoint_v3",)
mb_size = C.train.minibatch_size_schedule(40)

ts = C.train.training_session(trainer, mbs, mb_size,model_inputs_to_streams,
                              #checkpoint_config=checkpoint_config,
                              max_samples=80000
                              )

#for i in range(20000):

#    trainer.train_minibatch(get_data_batch())
    #model.save('cntk_run1_s1.model')
    #trainer.save_checkpoint("cntk_run1_s1.dnn")

        

    #evaluation_average = float(trainer.previous_minibatch_evaluation_average)
    #loss_average = float(trainer.previous_minibatch_loss_average)
    #return trainer
    #return evaluation_average, loss_average

In [ ]:
ts.train()
#trainer = make_trainer()
#trainer = trainer.restore_from_checkpoint("cntk_run1_s1.dnn")
#print trainer
#trainer = train_trainer(trainer)

In [ ]:
# try:
#     del ts
#     del trainer
#     del model
# except:
#     pass
# import gc
# gc.collect()

In [ ]:
# Learning rate per sample: 0.0005
#     0.741      0.741      0.612      0.612           500
#      1.16       1.37      0.377       0.26          1500
#     0.855      0.627      0.358      0.344          3500
#      0.72      0.601      0.313      0.273          7500
#     0.644      0.574      0.284      0.258         15500
#     0.606      0.568      0.269      0.255         31500
#      0.59      0.574      0.265       0.26         63500
#     0.571      0.552      0.259      0.254        127500
#     0.448      0.326      0.201      0.144        255500


In [ ]:
# model = trainer.model
# model.save('version1_10000.model')
# trainer.save_checkpoint("version1_10000.dnn")

In [ ]:
df = pd.read_csv(STAGE1_SAMPLE_SUBMISSION)
sols = pd.read_csv(STAGE1_SAMPLE_SUBMISSION_SOLUTIONS)


# testx = [np.squeeze(np.load(FEATURE_FOLDER+'%s.npy' % str(id))) for id in df['id'].tolist()]
xresnet = [np.squeeze(np.load(FEATURE_FOLDER+'%s.npy' % str(id))) for id in df['id'].tolist()]
xvgg = [np.squeeze(np.load('/a/data/mango/featuresvgg19/'+'%s.npy' % str(id))) for id in df['id'].tolist()]

testx=[]
for r,v in zip(xresnet,xvgg):
    v = v.reshape((v.shape[0],-1))
    testx.append(np.concatenate((r,v),axis=1))


    
    #I tried scaling down data and the summing... worked ok actually but not better than
    #other strategies
# xm = []
# for xi in testx:
#     xm.append(xi / xi.shape[0])
    
# testx = xm

In [ ]:
#model = C.ops.functions.Function.load("version1_10000.model")
model = trainer.model
pred_data_iter = grouper(40,testx)
preds = []
for data in pred_data_iter:
    pred = model.forward(list(data))
    _,pred = next(pred[1].iteritems())
    pred = scipy.delete(pred, 1, 1)
    preds.extend(list(np.squeeze(pred)))
pred = np.array(preds)
# _,pred = next(pred[1].iteritems())
# pred = scipy.delete(pred, 1, 1)  # delete second row of A
# pred = np.squeeze(pred)
#print (pred)
#np.savetxt("foo1.csv", pred, delimiter=",")

In [ ]:
truth = np.array(sols['cancer'])
#y_pred = np.clip(pred, (1e-15), 1 - (1e-15))
#print ((-(truth * np.log(y_pred)) - ((1 - truth)*np.log(1 - y_pred))))
                             
tprs = []
fprs = []
for threshold in np.arange(-100,100,0.05):
    size = float(truth.shape[0])
    predcancer = np.zeros_like(pred)
    predcancer[pred>threshold] = 1
    tpr_num = tpr_den = fpr_num = fpr_den = 0.
    for t,p in zip(truth,predcancer):
        if t==1 and p == 1:
            tpr_num += 1
        if t==1:
            tpr_den += 1
            
        if t==0 and p == 1:
            fpr_num += 1
        if t==0:
            fpr_den += 1
    tpr = tpr_num/tpr_den
    fpr = fpr_num/fpr_den
    tprs.append(tpr)
    fprs.append(fpr)
    #print (round(threshold,2),":",tpr,fpr)
    
#print (pred)

from sklearn import metrics

smax_pred = (C.softmax(pred).eval())
print(metrics.log_loss(truth,smax_pred))

print(metrics.auc(fprs,tprs))

In [ ]:
# from matplotlib import pyplot as plt


# plt.plot([0,1],[0,1])#straight line
# plt.plot(fprs,tprs)
# plt.savefig("versionT2highmean_100000.png")

In [ ]:
import cPickle
cPickle.dump((fprs,tprs), open('roc_data_7.pickle', 'wb')) 
#import pylab as pl

In [ ]:
#training loss: with high mean
#   0.0813     0.0033          0          0         40920

#training loss with regular mean
#   0.0413   0.000653          0          0         81880


In [ ]:
#with high mean:
# 9.96477122127
# 0.390319771059

#with regular mean
# 2.19896640348
# 0.633818589026

In [ ]:
# import cPickle
# (fprs,tprs) = cPickle.load(open('verstionT2normalmean.pickle', 'rb'))


In [ ]:
import numpy as np
import pylab as pl
import cPickle

(fprs0,tprs0) = cPickle.load(open('roc_data_0.pickle', 'rb'))
(fprs1,tprs1) = cPickle.load(open('roc_data_1.pickle', 'rb'))
(fprs2,tprs2) = cPickle.load(open('roc_data_2.pickle', 'rb'))
(fprs3,tprs3) = cPickle.load(open('roc_data_3.pickle', 'rb'))
(fprs4,tprs4) = cPickle.load(open('roc_data_4.pickle', 'rb'))
(fprs5,tprs5) = cPickle.load(open('roc_data_5.pickle', 'rb'))
(fprs6,tprs6) = cPickle.load(open('roc_data_6.pickle', 'rb'))
(fprs7,tprs7) = cPickle.load(open('roc_data_7.pickle', 'rb'))

pl.clf()
pl.plot(fprs0, tprs0, label='CNN + Mean (area = 0.64)')
pl.plot(fprs1, tprs1, label='CNN + RNN (area = 0.52)')
pl.plot(fprs2, tprs2, label='CNN + RNN + Mean (area = 0.62)')
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.0])
pl.xlabel('False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('CNN ROC curve comparison')
pl.legend(loc="lower right")
pl.savefig("figure2CNN_ROC.png")
pl.show()

pl.clf()
pl.plot(fprs6, tprs6, label='RNN + Mean (area = 0.61)')
pl.plot(fprs7, tprs7, label='RNN (area = 0.52)')
pl.plot(fprs0, tprs0, label='CNN + Mean (area = 0.64)')
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.0])
pl.xlabel('False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('RNN ROC curve comparison')
pl.legend(loc="lower right")
pl.savefig("figure3RNN_ROC.png")
pl.show()

In [ ]:
#CODE FOR MODELS ACTUALLY USED IN PAPER:

VERSION 0
        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        
        Fold(get_mean_rnn(),initial_state=(0,0,0)),

basic CNN + mean at end

loss: 2.09360234358
abc:  0.643212641533



VERION 1

        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Fold(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),

basic CNN + two rnn (last of seq taken)

3.0623921966
0.517668284186




VERSION 2

        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
        
        Fold(get_mean_rnn(),initial_state=(0,0,0)),

cnn + two run + mean

3.17747431249
0.622869229812



VERSION 3
        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
        
        Recurrence(get_mean_rnn(),initial_state=(0,0,0)),
        Fold(get_upper_mean_rnn2(),initial_state=(0,0,0),go_backwards=True),
9.49553715024
0.59219858156
cnn + rnn + highmean (v2)


VERSION 4
        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),

        
        
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
        
        Recurrence(get_mean_rnn(),initial_state=(0,0,0)),
        Fold(get_upper_mean_rnn(),initial_state=(0,0,0),go_backwards=True),
nan
0.599415204678
cnn + rnn + highmean (v1)


VERSION 5

        BatchNormalization(),
        
        C.ops.reshape(Dense(4500, activation=C.leaky_relu),(5,30,30)),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        
        Convolution((3,3), num_filters=5, pad=True, activation=C.leaky_relu),MaxPooling((3,3), strides=2),
        BatchNormalization(),
        

        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        C.ops.sequence.reduce_max(Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True)),
cnn + rnn + max
2.93672954327
0.626726390444

All previous trained 80,000 iterations
batch size 40


VERSION 6
        BatchNormalization(),
		Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),        
        Fold(get_mean_rnn(),initial_state=(0,0,0)),
2.77463137799
0.613039691427
rnn + mean


VERSION 7
		BatchNormalization(),
        Recurrence(LSTM(LSTM_dim, enable_self_stabilization=True)),
        Fold(LSTM(LSTM_dim, enable_self_stabilization=True),go_backwards=True),
2.69949306237
0.52967525196

rnn

